# Week 1 Analysis Notebook

This notebook is for all the work that was done in the first week of the project. This mainly includes introductory work, getting familiar with CAESAR, FSPS, H5py and messing around with the data.

The main file being used is m25n256_151.hdf5 which is the caesar catalog from a 25 Mpc/h volume with 2x256^3 particles, at z=0

In [2]:
import h5py 
import caesar
import yt
import numpy as np
import matplotlib.pyplot as plt

import os
os.environ['SPS_HOME'] = '/home/spujni/fsps'
import fsps

In [3]:
infile = ("/home/spujni/sim/m50n512/s50/Groups/m50n512_151.hdf5")
obj = caesar.load(infile)

In [4]:
galaxy_masses = [i.masses['total'] for i in obj.galaxies]
plt.hist(np.log10(galaxy_masses), bins=30)
plt.xlabel('Log Total Mass [Msun]')
plt.ylabel('Number of Galaxies')
plt.title('Galaxy Mass Distribution')

In [ ]:
fsps.list_filters()

In [ ]:
#file structure
with h5py.File(infile, 'r') as f:
    def print_structure(name, obj):
        print(name)
    
    print("HDF5 File Structure:")
    f.visititems(print_structure)
    
    print("\nTop-level keys:", list(f.keys()))

In [5]:
import astropy.units as u
from astropy.constants import c

with h5py.File(infile, 'r') as f:
    appmag_keys = [k for k in f['galaxy_data/dicts'].keys() if k.startswith('appmag.')]
    print("Found appmag keys:", appmag_keys)

    freqs = []
    fluxes = []
    labels = []

    for k in appmag_keys:
        mags = f[f'galaxy_data/dicts/{k}'][:]           # per-galaxy apparent mags (with dust)
        flux = 3631*10**(-mags / 2.5)                        # mag -> flux assuming AB
        filt = k.split('appmag.')[-1]                   

        try:
            fsps_filt = fsps.get_filter(filt)           # FSPS filter name
            wave = fsps_filt.lambda_eff * u.AA          # Angstrom
            freq = (c / wave).to(u.Hz).value            # ν = c / λ

            freqs.append(freq)
            fluxes.append(np.sum(flux))
            labels.append(filt)
        except Exception as e:
            print(f"Skipping {filt}: {e}")

freqs = np.array(freqs)
fluxes = np.array(fluxes)
labels = np.array(labels)
order = np.argsort(freqs)
freqs = freqs[order]
fluxes = fluxes[order]
labels = labels[order]
lambda_ = (c / freqs)

nu_f_nu = freqs * fluxes

plt.figure(figsize=(10, 6))
plt.loglog(lambda_, nu_f_nu, 'o', linewidth=2, markersize=6)
plt.xlabel(r'$\lambda$ (Å)')
plt.ylabel(r'$\nu F_\nu$')
plt.title('SED at z = 0')
plt.grid(True, alpha=0.3, which='both')
plt.tight_layout()
plt.show()